In [26]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.compute as pc
import pandas as pd
import os
import gc

import numpy as np
from sklearn.model_selection import train_test_split



In [27]:
import importlib
import utils
importlib.reload(utils)


print(dir(utils))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'column_filter', 'compute_null_proportions', 'compute_null_values', 'compute_stats', 'count_nb_lines', 'count_target_list', 'count_target_value', 'extract_division_names', 'filter_null_rows', 'np', 'os', 'pa', 'pc', 'pq', 'show_columns', 'siren_filter', 'var_filter', 'year_simplified']


In [28]:
#gestion mémoire  (l'équivalent de free en C)

gc.set_threshold(500, 5, 5)
gc.collect()

0

In [29]:
df_sans_details_financiers=pd.read_parquet("datasets_finaux/dataset_sans_details_financiers.parquet")
df_avec_details_financiers=pd.read_parquet("datasets_finaux/dataset_avec_details_financiers.parquet")

In [30]:
df_avec_details_financiers.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,...,Poids_BFR_exploitation_sur_CA_jours_std,Rotation_des_stocks_jours_tcam,Rotation_des_stocks_jours_moyen,Rotation_des_stocks_jours_std,Credit_clients_Jours_tcam,Credit_clients_Jours_moyen,Credit_clients_Jours_std,Credit_fournisseurs_Jours_tcam,Credit_fournisseurs_Jours_moyen,Credit_fournisseurs_Jours_std
0,480086784,2015,-1,0,3,1,2023,1,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,43.591,0.000000e+00
1,514943323,2009,-1,0,3,1,2023,1,5710,7022,...,0.000000e+00,0.0,22.369,0.0,0.0,0.000,0.000000e+00,0.0,33.286,0.000000e+00
2,519715593,2010,2,2023,2,1,2023,1,5710,6209,...,3.552714e-15,0.0,5.602,0.0,0.0,39.735,0.000000e+00,0.0,42.464,0.000000e+00
3,520267881,2010,-1,0,2,1,2023,0,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,52.993,0.000000e+00
4,522761139,2010,21,2023,5,1,2023,1,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,78.778,0.000000e+00,0.0,21.579,3.552714e-15
5,522807981,2010,12,2023,3,1,2023,1,5710,6399,...,0.000000e+00,0.0,37.858,0.0,0.0,70.764,0.000000e+00,0.0,152.765,0.000000e+00
6,523617173,2010,1,2023,6,1,2023,1,5710,6201,...,1.421085e-14,0.0,7.751,0.0,0.0,354.581,5.684342e-14,0.0,102.188,0.000000e+00
7,524157872,2010,-1,0,2,0,0,0,5710,7022,...,0.000000e+00,0.0,0.000,0.0,0.0,0.000,0.000000e+00,0.0,73.204,0.000000e+00
8,524607868,2010,-1,0,4,1,2023,1,5710,6201,...,0.000000e+00,0.0,0.000,0.0,0.0,23.357,0.000000e+00,0.0,0.813,1.110223e-16
9,525268975,2010,-1,0,4,1,2023,0,5710,7022,...,0.000000e+00,0.0,104.706,0.0,0.0,3.013,0.000000e+00,0.0,70.758,0.000000e+00


In [31]:
df_sans_details_financiers.head(10)

,siren,dateCreationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,etatAdministratifUniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,nbEtablissements,aDéclaréBilan(s)FinancierSur3Ans
0,5580436,1955,-1,0,7,1,2023,1,5710,7022,-1,0,3,0
1,24074924,2002,-1,0,2,2,2023,1,5710,6311,-1,0,1,0
2,35650167,1956,32,2023,6,2,2023,1,5710,2611,0,0,6,0
3,36420040,1964,12,2023,8,1,2023,1,5710,7219,-1,0,16,0
4,46750014,1967,12,2023,5,1,2023,1,5710,7211,-1,0,3,0
5,55811095,1955,-1,0,7,0,0,0,5710,7022,-1,0,3,0
6,57801136,1957,-1,0,5,0,0,0,5710,2651,0,0,1,0
7,62804414,1962,-1,0,9,1,2023,0,5710,7022,-1,0,5,0
8,64502636,1964,-1,0,12,0,0,0,5710,6202,-1,0,50,0
9,67201087,1967,-1,0,7,2,2023,1,5710,7022,-1,0,2,0


In [32]:
df_sans_details_financiers=df_sans_details_financiers.drop(["siren"],axis=1)
df_avec_details_financiers=df_avec_details_financiers.drop(["siren"],axis=1)

In [33]:
df_sans_details_financiers.sort_values('dateCreationUniteLegale').head(10)
df_sans_details_financiers.sort_values('dateCreationUniteLegale').head(50)

print((df_sans_details_financiers['dateCreationUniteLegale']==1900).sum())

47


In [34]:
df_sans_details_financiers['dateCreationUniteLegale'].describe()

count    285441.000000
mean       2018.136519
std           6.964259
min        1900.000000
25%        2016.000000
50%        2020.000000
75%        2023.000000
max        2026.000000
Name: dateCreationUniteLegale, dtype: float64

In [35]:
df_avec_details_financiers['dateCreationUniteLegale'].describe()

count    36388.000000
mean      2018.387765
std          2.830421
min       2009.000000
25%       2016.000000
50%       2018.000000
75%       2021.000000
max       2025.000000
Name: dateCreationUniteLegale, dtype: float64

In [36]:
print((df_sans_details_financiers['nbEtablissements']==0).sum())
print((df_avec_details_financiers['nbEtablissements']== 0).sum())



0
0


In [37]:
print((df_sans_details_financiers['dateCreationUniteLegale']>= 2009).sum())
print((df_avec_details_financiers['dateCreationUniteLegale']>= 2009).sum())

264249
36388


In [38]:
df_avec_details_financiers=df_avec_details_financiers[df_avec_details_financiers['dateCreationUniteLegale']>=2009]
df_sans_details_financiers=df_sans_details_financiers[df_sans_details_financiers['dateCreationUniteLegale']>=2009]


In [39]:
proportion_survie_sans_details = (df_sans_details_financiers['etatAdministratifUniteLegale'] == 1).mean() 
print(proportion_survie_sans_details)


proportion_survie_avec_details = (df_avec_details_financiers['etatAdministratifUniteLegale'] == 1).mean() 
print(proportion_survie_avec_details)

0.7465685773645312
0.7539848301637903


In [40]:
print(df_avec_details_financiers['dateCreationUniteLegale'].describe())
print(df_sans_details_financiers['dateCreationUniteLegale'].describe())

count    36388.000000
mean      2018.387765
std          2.830421
min       2009.000000
25%       2016.000000
50%       2018.000000
75%       2021.000000
max       2025.000000
Name: dateCreationUniteLegale, dtype: float64
count    264249.000000
mean       2019.576596
std           4.200730
min        2009.000000
25%        2017.000000
50%        2020.000000
75%        2023.000000
max        2026.000000
Name: dateCreationUniteLegale, dtype: float64


In [41]:
def split(dataset,seed=42,yes_validation=False):
    x=dataset.drop(["etatAdministratifUniteLegale"],axis=1)
    y=dataset["etatAdministratifUniteLegale"]
    if(yes_validation==True): #crée un validation set
        x_train, x_temp, y_train, y_temp = train_test_split( x, y, test_size=0.3, random_state=seed,stratify=y ) 
        x_validation, x_test, y_validation, y_test = train_test_split( x_temp, y_temp, test_size=0.5, random_state=seed,stratify=y_temp )
        return x_train,x_test,y_train,y_test,x_validation, y_validation
    else:
        x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, random_state=seed,stratify=y )
        return x_train,x_test,y_train,y_test


In [42]:
x_train,x_test,y_train,y_test=split(df_avec_details_financiers,seed=42) #possibilité de changer le dataset

#Avec validation set
#x_train,x_test,y_train,y_test,x_validation, y_validation=split(df_avec_details_financiers,seed=42,yes_validation=True)


In [43]:
x_test.isna().sum()

dateCreationUniteLegale               0
trancheEffectifsUniteLegale           0
anneeEffectifsUniteLegale             0
nombrePeriodesUniteLegale             0
categorieEntreprise                   0
                                   ... 
Credit_clients_Jours_moyen         3150
Credit_clients_Jours_std           3150
Credit_fournisseurs_Jours_tcam        0
Credit_fournisseurs_Jours_moyen    1628
Credit_fournisseurs_Jours_std      1628
Length: 68, dtype: int64

In [44]:
x_train.isna().sum()

dateCreationUniteLegale               0
trancheEffectifsUniteLegale           0
anneeEffectifsUniteLegale             0
nombrePeriodesUniteLegale             0
categorieEntreprise                   0
                                   ... 
Credit_clients_Jours_moyen         7380
Credit_clients_Jours_std           7380
Credit_fournisseurs_Jours_tcam        0
Credit_fournisseurs_Jours_moyen    3811
Credit_fournisseurs_Jours_std      3811
Length: 68, dtype: int64

In [45]:
cols_to_fill = [col for col in x_train.columns 
                if col.endswith(('_moyen', '_std'))]

medians_train = x_train[cols_to_fill].median()



for col in cols_to_fill:
    x_train[col] = x_train[col].fillna(medians_train[col])


for col in cols_to_fill:
    x_test[col] = x_test[col].fillna(medians_train[col]) #pour éviter la triche
 
#Avec validation set (à décommenter si usage de validation set)
# for col in cols_to_fill:
#     x_validation[col] = x_validation[col].fillna(medians_train[col]) #pour éviter la triche
 


In [46]:
x_test.isna().sum().sum()

np.int64(0)

In [47]:

x_train.isna().sum().sum()

np.int64(0)

In [48]:
y_train.isna().sum()

np.int64(0)

In [49]:
y_test.isna().sum()

np.int64(0)

In [50]:
# #LIBERER MEMOIRE

# del df_avec_details_financiers
# del df_sans_details_financiers

# gc.collect()